# Avaliação Sprint 3

## Import das bibliotecas necessárias

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
from pymongo import MongoClient
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt


## Conexão com o banco de dados Mongo

In [3]:
client = MongoClient("localhost", 27017)
artigos_db = client["Artigos"]
artigos = artigos_db["Coleção_artigos"]

## Fazendo o scrapping dos artigos

In [5]:
urls = [
    "https://www1.folha.uol.com.br/banco-de-dados/2021/05/1921-transporte-de-cavalos-de-corrida-burla-proibicao-em-sao-paulo.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/06/1921-apos-sucesso-nos-eua-pianista-guiomar-novaes-chega-a-sao-paulo.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/05/1921-missao-italiana-vem-ao-brasil-buscar-ajuda-para-mutilados-na-guerra.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/05/1971-franca-aprova-entrada-do-reino-unido-no-mercado-comum-europeu.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/05/1921-empresa-de-campinas-vai-comecar-a-vender-carne-fresca-a-paulistanos.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/05/1921-teatro-polytheama-em-ribeirao-preto-e-destruido-por-incendio.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/04/1921-tracado-de-estrada-que-ligara-limeira-a-araras-e-estudado.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/03/1971-guerra-civil-estoura-no-paquistao-oriental-que-deseja-independencia.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/03/1921-estrada-vai-facilitar-ligacao-de-sao-paulo-com-triangulo-mineiro-e-goias.shtml",
    "https://www1.folha.uol.com.br/banco-de-dados/2021/02/1921-autoridade-do-reino-unido-reafirma-luta-contra-movimento-irlandes.shtml"
] # lista das urls dos artigos



In [45]:
lista_artigos = []
for url in urls:
    response = urlopen(url)
    html = response.read().decode('utf8')

    soup = BeautifulSoup(html, 'html.parser')

    paragrafos_artigo = soup.find('div', {'class': 'c-news__body'}).findAll('p')
    # Retirando as tags html dos artigos
    # As tags html, são substituidas por uma string vazia
    paragrafos_artigo = [str(paragrafo).replace('<p>','') for paragrafo in paragrafos_artigo]
    paragrafos_artigo = [str(paragrafo).replace('</p>','') for paragrafo in paragrafos_artigo]

    lista_artigos.append({'Paragrafos': paragrafos_artigo})


In [46]:
lista_artigos[0]

{'Paragrafos': ['Está sendo burlada a proibição de transportar animais nas estradas que convergem a São Paulo.',
  'Dias depois do aparecimento da peste bovina nos arredores da capital, as autoridades sanitárias tinham lembrado o governo da necessidade de estabelecer essa proibição.',
  'Apesar da medida, jornais noticiaram a ida e vinda de cavalos de corrida entre a capital e Santos. A fim de mascarar a manobra, os animais são enviados para São Bernardo e de lá são transportados para a outra cidade.',
  'Isso ocorre como se a transmissão da peste não fosse possível nas patas de cavalos ingleses, que custam caro.']}

## Salvando os artigos no MongoDB

In [47]:
artigos.insert_one({'Artigos': lista_artigos})


## Criando um dataset com os paragrafos dos artigos

In [48]:
paragrafos_artigos = artigos.find({'Artigos': lista_artigos}) # Retirando os paragrafos direto do mongo
for paragrafo in paragrafos_artigos:
    df_paragrafos = pd.DataFrame(paragrafo['Artigos'])
df_paragrafos.head()

,Paragrafos
0,"[Está sendo burlada a proibição de transportar animais nas estradas que convergem a São Paulo., Dias depois do aparecimento da peste bovina nos arredores da capital, as autoridades sanitárias tinham lembrado o governo da necessidade de estabelecer essa proibição., Apesar da medida, jornais noticiaram a ida e vinda de cavalos de corrida entre a capital e Santos. A fim de mascarar a manobra, os animais são enviados para São Bernardo e de lá são transportados para a outra cidade., Isso ocorre c..."
1,"[A pianista Guiomar Novaes chegou a São Paulo nesta quinta-feira (2). Ela veio em um trem noturno do Rio de Janeiro, após regressar de uma viagem triunfal aos Estados Unidos, onde foi considerada, por vários críticos, a maior virtuose do mundo., Guiomar Novaes tinha viajado para os EUA em dezembro de 1920 para realizar uma série de concertos, mas foi tanto o seu sucesso em Nova York e em outras cidades que a sequência de apresentações teve que ser ampliada., Ela tem recebido propostas para t..."
2,"[A missão italiana de mutilados na guerra chegou ao Rio de Janeiro, nesta quinta-feira (25), vinda de Buenos Aires, onde buscou angariar donativos., O grupo, chefiado pelo tenente Carlo Delcroix, veio ao Brasil com o mesmo objetivo. Por meio de conferências e de outras atividades, vai tentar arrecadar fundos para minorar o sofrimento dos feridos no conflito., Elevado número de membros da colônia italiana no Rio compareceu ao desembarque da missão. Várias festas devem ser feitas., No início d..."
3,"[A França deu o seu “sim” para o ingresso do Reino Unido no Mercado Comum Europeu, depois de dez horas de conversações entre o presidente francês, Georges Pompidou, e o primeiro-ministro britânico, Edward Heath., A entrada pode ser concretizada já no próximo mês., O comunicado, com a declaração conjunta, afirma que muitos pontos de vista são suficientes próximos que não há pessimismo., O Mercado Comum Europeu é formado pela Alemanha Ocidental, Bélgica, Itália, Holanda, Luxemburgo e França., ..."
4,"[Uma empresa particular, de Campinas, vai passar a vender carne fresca para os moradores da cidade de São Paulo a partir desta terça-feira (10)., Por causa da peste bovina, a população da capital vinha sendo abastecida com carnes apenas congeladas ou frigorificadas. No entanto, muitos não gostam de comprar assim e reclamam da qualidade. Além disso, o preço está elevado., O fato é que o paulistano acabou reduzindo o consumo nesse período., Isso induziu a empresa Figueiredo &amp; Cia a prepara..."


In [49]:
nlp = spacy.load("pt_core_news_sm")
doc = nlp(str(df_paragrafos.Paragrafos))